In [1]:
import pandas as pd
import os
import sys
import numpy as np

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/society_economy/economic_health/bureau_labor_statistics/employment_hachman_index/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

aws_dir = '1_pull_data/society_economy/economic_health/'
pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'employment_data_hachman_subset.csv'
Saved DataFrame as 'ACSDT5Y2022.B19083-Column-Metadata.csv'
Saved DataFrame as 'ACSDT5Y2022.B19083-Data.csv'
Saved DataFrame as 'ACSST5Y2022.S1901-Column-Metadata.csv'


c:\Users\jesse\eagle\carb-climate-index-4\scripts\data_metric_calc\../..\scripts\utils\file_helpers.py:100: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'ACSST5Y2022.S1901-Data.csv'


c:\Users\jesse\eagle\carb-climate-index-4\scripts\data_metric_calc\../..\scripts\utils\file_helpers.py:100: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'allhlcn22.csv'
Saved DataFrame as 'data_layout.csv'


In [3]:
hachman_data = pd.read_csv('employment_data_hachman_subset.csv')
gini_index_data = pd.read_csv('ACSDT5Y2022.B19083-Data.csv')
median_income_data = pd.read_csv('ACSST5Y2022.S1901-Data.csv')


C:\Users\jesse\AppData\Local\Temp\ipykernel_24136\2131310057.py:3: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  median_income_data = pd.read_csv('ACSST5Y2022.S1901-Data.csv')


In [4]:
county_tract = "s3://ca-climate-index/0_map_data/ca_tract_county_population_2022.csv"
ca_county_tract = pd.read_csv(county_tract)
ca_county_tract = ca_county_tract.rename(columns={'Census Tract': 'GEO_ID'})
ca_county_tract = ca_county_tract.drop(columns={'Unnamed: 0', 'COUNTYFP', 'County', 'Total Population 2021'})
ca_county_tract

,GEO_ID
0,6085504321
1,6085504410
2,6085507003
3,6085507004
4,6085502204
...,...
9124,6059001303
9125,6059001304
9126,6059001401
9127,6013367200


Gini

In [5]:
gini_index_data_cleaned = gini_index_data.drop(columns={'Unnamed: 4', 'B19083_001M', 'NAME'})
gini_index_data_cleaned = gini_index_data_cleaned.rename(columns={'B19083_001E':'est_gini_index'})
gini_index_data_cleaned = gini_index_data_cleaned[1:]
gini_index_data_cleaned['GEO_ID'] = gini_index_data_cleaned['GEO_ID'].str[10:]

gini_index_data_cleaned

,GEO_ID,est_gini_index
1,6001400100,0.4228
2,6001400200,0.4084
3,6001400300,0.4615
4,6001400400,0.5063
5,6001400500,0.4571
...,...,...
9125,6115040902,0.4252
9126,6115041001,0.5024
9127,6115041002,0.4875
9128,6115041101,0.4072


In [6]:
# Convert the 'GEO_ID' column in ca_county_tract DataFrame to string data type
ca_county_tract['GEO_ID'] = ca_county_tract['GEO_ID'].astype(str)

# Now, you can proceed with merging
gini_merged_data = pd.merge(gini_index_data_cleaned, ca_county_tract, on='GEO_ID', how='right')

# View the merged DataFrame
gini_merged_data

,GEO_ID,est_gini_index
0,6085504321,0.4197
1,6085504410,0.4309
2,6085507003,0.5357
3,6085507004,0.4793
4,6085502204,0.4010
...,...,...
9124,6059001303,0.3590
9125,6059001304,0.3582
9126,6059001401,0.4594
9127,6013367200,0.4130


Median

In [7]:
columns_to_keep = ['GEO_ID', 'S1901_C01_012E']

# Drop all columns except the specified ones
median_income_data_cleaned = median_income_data.drop(columns=[col for col in median_income_data.columns if col not in columns_to_keep])

median_income_data_cleaned = median_income_data_cleaned.rename(columns={'S1901_C01_012E':'est_median_income_dollars'})
median_income_data_cleaned = median_income_data_cleaned[1:]
median_income_data_cleaned['GEO_ID'] = median_income_data_cleaned['GEO_ID'].str[10:]
median_income_data_cleaned

,GEO_ID,est_median_income_dollars
1,6001400100,234236
2,6001400200,225500
3,6001400300,164000
4,6001400400,158836
5,6001400500,95078
...,...,...
9125,6115040902,54265
9126,6115041001,67321
9127,6115041002,102534
9128,6115041101,37018


In [8]:
# Convert the 'GEO_ID' column in ca_county_tract DataFrame to string data type
ca_county_tract['GEO_ID'] = ca_county_tract['GEO_ID'].astype(str)

# Now, you can proceed with merging
median_merged_data = pd.merge(median_income_data_cleaned, ca_county_tract, on='GEO_ID', how='right')

# View the merged DataFrame
median_merged_data

,GEO_ID,est_median_income_dollars
0,6085504321,172857
1,6085504410,143173
2,6085507003,240179
3,6085507004,198306
4,6085502204,67552
...,...,...
9124,6059001303,93363
9125,6059001304,68984
9126,6059001401,76319
9127,6013367200,90941


Hachman

$$
HI = \frac{1}{Σ_j \cdot \left(\frac{EMPcounty_j}{EMPstate_j}\right) \cdot EMPScounty_j}
$$


In [51]:
hachman_data.head()

,Area\nCode,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,Area,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,6000,6,0,0,10,2022,A,State,California,California -- Statewide,Total Covered,"10 Total, all industries",NaN,"1,706,672","17,903,539","1,511,706,499,616","1,624","84,436",1.00,1.00
1,6000,6,0,1,10,2022,A,State,California,California -- Statewide,Federal Government,"10 Total, all industries",NaN,"3,286","247,795","23,655,167,888","1,836","95,463",0.73,0.62
2,6000,6,0,2,10,2022,A,State,California,California -- Statewide,State Government,"10 Total, all industries",NaN,"13,784","492,218","47,749,398,000","1,866","97,009",0.91,1.00
3,6000,6,0,3,10,2022,A,State,California,California -- Statewide,Local Government,"10 Total, all industries",NaN,"19,344","1,724,972","135,804,850,033","1,514","78,729",1.04,1.11
4,6000,6,0,5,10,2022,A,State,California,California -- Statewide,Private,"10 Total, all industries",NaN,"1,670,259","15,438,555","1,304,497,083,695","1,625","84,496",1.01,1.00


Taking a look at the entries within area
* separating California entries as we will need that data also
* getting rid of all 'County, California' portions of each entry

In [10]:
# Get unique entries in 'Column1'
unique_entries = hachman_data['Area'].unique()

print(unique_entries)

['California -- Statewide' 'Alameda County, California'
 'Alpine County, California' 'Amador County, California'
 'Butte County, California' 'Calaveras County, California'
 'Colusa County, California' 'Contra Costa County, California'
 'Del Norte County, California' 'El Dorado County, California'
 'Fresno County, California' 'Glenn County, California'
 'Humboldt County, California' 'Imperial County, California'
 'Inyo County, California' 'Kern County, California'
 'Kings County, California' 'Lake County, California'
 'Lassen County, California' 'Los Angeles County, California'
 'Madera County, California' 'Marin County, California'
 'Mariposa County, California' 'Mendocino County, California'
 'Merced County, California' 'Modoc County, California'
 'Mono County, California' 'Monterey County, California'
 'Napa County, California' 'Nevada County, California'
 'Orange County, California' 'Placer County, California'
 'Plumas County, California' 'Riverside County, California'
 'Sacramento 

In [11]:
hachman_data_cleaned = hachman_data[['Area', 'Industry', 'Annual Average Employment']]

# Remove any mention of 'county' within the legalAgencyName column
hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' -- Statewide', '', case=False)
hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' County, California', '', case=False)

# create a Boolean mask for the rows to remove
mask = hachman_data_cleaned['Industry'] == '10 Total, all industries'
# select all rows except the ones that contain '10 Total, all industries' which are state and county employment totals, not needed
hachman_data_cleaned = hachman_data_cleaned[~mask]

unique_entries = hachman_data_cleaned['Area'].unique()

C:\Users\jesse\AppData\Local\Temp\ipykernel_24136\4121247355.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' -- Statewide', '', case=False)
C:\Users\jesse\AppData\Local\Temp\ipykernel_24136\4121247355.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' County, California', '', case=False)


In [12]:
hachman_data_cleaned

,Area,Industry,Annual Average Employment
5,California,101 Goods-producing,"2,685,113"
6,California,1011 Natural resources and mining,"436,965"
7,California,1012 Construction,"912,359"
8,California,1013 Manufacturing,"1,335,789"
9,California,102 Service-providing,"12,753,442"
...,...,...,...
1069,"Unknown Or Undefined, California",1024 Professional and business services,"236,341"
1070,"Unknown Or Undefined, California",1025 Education and health services,"30,314"
1071,"Unknown Or Undefined, California",1026 Leisure and hospitality,"10,276"
1072,"Unknown Or Undefined, California",1027 Other services,0


In [13]:
unique_entries = hachman_data_cleaned['Industry'].unique()
unique_entries

array(['101 Goods-producing', '1011 Natural resources and mining',
       '1012 Construction', '1013 Manufacturing', '102 Service-providing',
       '1021 Trade, transportation, and utilities', '1022 Information',
       '1023 Financial activities',
       '1024 Professional and business services',
       '1025 Education and health services',
       '1026 Leisure and hospitality', '1027 Other services',
       '1029 Unclassified'], dtype=object)

Running our county function filter to separate CA counties and other counties

In [14]:
filtered_hachman_data, omitted_data = filter_counties(hachman_data_cleaned, 'Area')

print('Counties kept:', len(filtered_hachman_data))
print('Omitted data entries:', len(omitted_data))

Counties kept: 749
Omitted data entries: 26


In [15]:
filtered_hachman_data.rename(columns={'Annual Average Employment': 'EMP_COUNTY', 'Area':'County'}, inplace=True)
filtered_hachman_data


C:\Users\jesse\AppData\Local\Temp\ipykernel_24136\1468599509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hachman_data.rename(columns={'Annual Average Employment': 'EMP_COUNTY', 'Area':'County'}, inplace=True)


,County,Industry,EMP_COUNTY
23,Alameda,101 Goods-producing,"146,054"
24,Alameda,1011 Natural resources and mining,959
25,Alameda,1012 Construction,"47,512"
26,Alameda,1013 Manufacturing,"97,583"
27,Alameda,102 Service-providing,"536,253"
...,...,...,...
1051,Yuba,1024 Professional and business services,"1,265"
1052,Yuba,1025 Education and health services,"3,615"
1053,Yuba,1026 Leisure and hospitality,"1,893"
1054,Yuba,1027 Other services,194


# Looking at the other entries, will isolate for just California data

In [16]:
omitted_data

,Area,Industry,Annual Average Employment
5,California,101 Goods-producing,"2,685,113"
6,California,1011 Natural resources and mining,"436,965"
7,California,1012 Construction,"912,359"
8,California,1013 Manufacturing,"1,335,789"
9,California,102 Service-providing,"12,753,442"
10,California,"1021 Trade, transportation, and utilities","3,121,930"
11,California,1022 Information,"605,857"
12,California,1023 Financial activities,"841,564"
13,California,1024 Professional and business services,"2,860,674"
14,California,1025 Education and health services,"2,854,971"


In [17]:
mask = omitted_data['Area'] == 'Unknown Or Undefined, California'
california_employ_data = omitted_data[~mask]
california_employ_data.rename(columns={'Annual Average Employment': 'EMP_CA', 'Area': 'State'}, inplace=True)

california_employ_data

C:\Users\jesse\AppData\Local\Temp\ipykernel_24136\3214061953.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  california_employ_data.rename(columns={'Annual Average Employment': 'EMP_CA', 'Area': 'State'}, inplace=True)


,State,Industry,EMP_CA
5,California,101 Goods-producing,"2,685,113"
6,California,1011 Natural resources and mining,"436,965"
7,California,1012 Construction,"912,359"
8,California,1013 Manufacturing,"1,335,789"
9,California,102 Service-providing,"12,753,442"
10,California,"1021 Trade, transportation, and utilities","3,121,930"
11,California,1022 Information,"605,857"
12,California,1023 Financial activities,"841,564"
13,California,1024 Professional and business services,"2,860,674"
14,California,1025 Education and health services,"2,854,971"


In [18]:
filtered_hachman_data

,County,Industry,EMP_COUNTY
23,Alameda,101 Goods-producing,"146,054"
24,Alameda,1011 Natural resources and mining,959
25,Alameda,1012 Construction,"47,512"
26,Alameda,1013 Manufacturing,"97,583"
27,Alameda,102 Service-providing,"536,253"
...,...,...,...
1051,Yuba,1024 Professional and business services,"1,265"
1052,Yuba,1025 Education and health services,"3,615"
1053,Yuba,1026 Leisure and hospitality,"1,893"
1054,Yuba,1027 Other services,194


Merge the two datasets together based on industry so we have county employment and state employment per industry

In [19]:
# Merge county industry employment with state industry employment
merged_data = filtered_hachman_data.merge(california_employ_data, on='Industry')
merged_data

,County,Industry,EMP_COUNTY,State,EMP_CA
0,Alameda,101 Goods-producing,"146,054",California,"2,685,113"
1,Alpine,101 Goods-producing,13,California,"2,685,113"
2,Amador,101 Goods-producing,"1,865",California,"2,685,113"
3,Butte,101 Goods-producing,"11,325",California,"2,685,113"
4,Calaveras,101 Goods-producing,"1,500",California,"2,685,113"
...,...,...,...,...,...
744,Tulare,1029 Unclassified,5,California,"1,970"
745,Tuolumne,1029 Unclassified,0,California,"1,970"
746,Ventura,1029 Unclassified,17,California,"1,970"
747,Yolo,1029 Unclassified,11,California,"1,970"


Function to check a df's results per county

In [52]:
# Define a function to display data for a specific county
def display_county_data(df, county_name):
    county_data = df[df['County'] == county_name]
    if county_data.empty:
        print(f"No data found for {county_name}")
    else:
        print(f"Data for {county_name}:")
        display(county_data)
        

In [39]:
display_county_data(merged_data, 'Alameda')

Data for Alameda:


,County,Industry,EMP_COUNTY,State,EMP_CA
0,Alameda,101 Goods-producing,"146,054.00",California,"2,685,113.00"
58,Alameda,1011 Natural resources and mining,959.00,California,"436,965.00"
115,Alameda,1012 Construction,"47,512.00",California,"912,359.00"
173,Alameda,1013 Manufacturing,"97,583.00",California,"1,335,789.00"
231,Alameda,102 Service-providing,"536,253.00",California,"12,753,442.00"
289,Alameda,"1021 Trade, transportation, and utilities","135,720.00",California,"3,121,930.00"
347,Alameda,1022 Information,"18,269.00",California,"605,857.00"
405,Alameda,1023 Financial activities,"26,122.00",California,"841,564.00"
463,Alameda,1024 Professional and business services,"133,973.00",California,"2,860,674.00"
521,Alameda,1025 Education and health services,"128,076.00",California,"2,854,971.00"


Create new column containing the product of a counties employment divided by the states employment (then multiplied by county employment) per industry

In [21]:
denominator = pd.DataFrame(merged_data)
merged_data['EMP_COUNTY'] = merged_data['EMP_COUNTY'].str.replace(',', '').astype(float)
merged_data['EMP_CA'] = merged_data['EMP_CA'].str.replace(',', '').astype(float)

denominator['emp_county_div_emp_ca'] = (merged_data['EMP_COUNTY'] / merged_data['EMP_CA']) * merged_data['EMP_COUNTY']
denominator

,County,Industry,EMP_COUNTY,State,EMP_CA,emp_county_div_emp_ca
0,Alameda,101 Goods-producing,"146,054",California,"2,685,113",7944.459289
1,Alpine,101 Goods-producing,13,California,"2,685,113",0.000063
2,Amador,101 Goods-producing,"1,865",California,"2,685,113",1.295374
3,Butte,101 Goods-producing,"11,325",California,"2,685,113",47.765448
4,Calaveras,101 Goods-producing,"1,500",California,"2,685,113",0.837954
...,...,...,...,...,...,...
744,Tulare,1029 Unclassified,5,California,"1,970",0.012690
745,Tuolumne,1029 Unclassified,0,California,"1,970",0.000000
746,Ventura,1029 Unclassified,17,California,"1,970",0.146701
747,Yolo,1029 Unclassified,11,California,"1,970",0.061421


In [40]:
display_county_data(denominator, 'Alameda')

Data for Alameda:


,County,Industry,EMP_COUNTY,State,EMP_CA,emp_county_div_emp_ca
0,Alameda,101 Goods-producing,"146,054",California,"2,685,113","7,944.46"
58,Alameda,1011 Natural resources and mining,959,California,"436,965",2.10
115,Alameda,1012 Construction,"47,512",California,"912,359","2,474.23"
173,Alameda,1013 Manufacturing,"97,583",California,"1,335,789","7,128.70"
231,Alameda,102 Service-providing,"536,253",California,"12,753,442","22,548.21"
289,Alameda,"1021 Trade, transportation, and utilities","135,720",California,"3,121,930","5,900.17"
347,Alameda,1022 Information,"18,269",California,"605,857",550.88
405,Alameda,1023 Financial activities,"26,122",California,"841,564",810.82
463,Alameda,1024 Professional and business services,"133,973",California,"2,860,674","6,274.31"
521,Alameda,1025 Education and health services,"128,076",California,"2,854,971","5,745.58"


Group by the county and sum the new column so we have the denominator portion of the Hachman calculation

In [35]:
pd.set_option('display.float_format', '{:,.2f}'.format)

denominator_county_sum = denominator.groupby('County')['emp_county_div_emp_ca'].sum().reset_index()
denominator_county_sum.head()

,County,emp_county_div_emp_ca
0,Alameda,"63,030.06"
1,Alpine,0.06
2,Amador,8.31
3,Butte,569.78
4,Calaveras,8.81


Make a new column containing the results from 1/Hachman calculation denominator

In [36]:
hachman_calculation = pd.DataFrame(denominator_county_sum)
hachman_calculation['hachman_index_value'] = 1 / denominator_county_sum['emp_county_div_emp_ca']

### Final answer in hachman_calculation df, the numbers are way too small

In [50]:
hachman_calculation

,County,emp_county_div_emp_ca,hachman_index_value
0,Alameda,"63,030.06",0.00
1,Alpine,0.06,16.24
2,Amador,8.31,0.12
3,Butte,569.78,0.00
4,Calaveras,8.81,0.11
5,Colusa,20.87,0.05
6,Contra Costa,"14,106.69",0.00
7,Del Norte,3.19,0.31
8,El Dorado,321.04,0.00
9,Fresno,"18,262.25",0.00


### The code below is running the calculation steps together

In [49]:
merged_data = filtered_hachman_data.merge(california_employ_data, on='Industry')
denominator = pd.DataFrame(merged_data)
merged_data['EMP_COUNTY'] = merged_data['EMP_COUNTY'].str.replace(',', '').astype(float)
merged_data['EMP_CA'] = merged_data['EMP_CA'].str.replace(',', '').astype(float)

denominator['emp_county_div_emp_ca'] = (merged_data['EMP_COUNTY'] / merged_data['EMP_CA']) * merged_data['EMP_COUNTY']

pd.set_option('display.float_format', '{:,.2f}'.format)

denominator_county_sum = denominator.groupby('County')['emp_county_div_emp_ca'].sum().reset_index()
denominator_county_sum.head()

hachman_calculation = pd.DataFrame(denominator_county_sum)
hachman_calculation['hachman_index_value'] = 1 / denominator_county_sum['emp_county_div_emp_ca']
hachman_calculation


,County,emp_county_div_emp_ca,hachman_index_value
0,Alameda,"63,030.06",0.00
1,Alpine,0.06,16.24
2,Amador,8.31,0.12
3,Butte,569.78,0.00
4,Calaveras,8.81,0.11
5,Colusa,20.87,0.05
6,Contra Costa,"14,106.69",0.00
7,Del Norte,3.19,0.31
8,El Dorado,321.04,0.00
9,Fresno,"18,262.25",0.00
